In [6]:
import os
import unicodedata
import string
import pandas as pd
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer

In [97]:
 
#################### EXTRACTION ####################

def extractionSeries(chemin_repertoire_racine):
    """
    Fonction qui va extraire les titres des différents séries.
    
    param : chemin_repertoire_racine (string) -> chemin vers le repertoires racine qui contient.
    Un ensemble de sous repertoires et fichier texte qui correspond a une serie avec le titre de ces episodes.
    return : liste1[liste2[string]]  -> une liste1 ou chaque element de la liste1 correspond a une serie.
    La serie est une liste2[string] ou chaque element de cette liste correspond a un titre d'un episode de cette serie.
    """

    liste = []
    #Parcours des differentes series
    for serie in os.listdir(chemin_repertoire_racine) :
        chemin = chemin_repertoire_racine + "/" + serie
        serie = normaliseMot(listeFonctionNormaliseTitreSerie(),serie)
        #serie = filtreMot(listeFonctionNormaliseTitreSerie(),serie)
        liste.append( serie )
        #On va extraire les differents d'episodes de chaque serie
        liste += extractionDonnerSeries(chemin)
    
    #S'applique a la fin
    liste = filtreChaineVide(liste)
    return liste


def extractionDonnerSeries(chemin_sous_repertoire_racine) :
    """
    Fonction qui va extraire les titres des différents episodes des differentes saison d'une series.
    
    param : chemin_sous_repertoire_racine (string) -> chemin vers l'un des sous repertoires du dossier racine qui     contient un ensemble de sous repertoires et fichier texte qui correspond a une serie avec le titre de ces episodes.
    return : liste[string] -> liste de tous les titres des épisodes d'une série
    """
    mot = []

    for root, directories, files in os.walk(chemin_sous_repertoire_racine):  
       
        #Parcours l'ensemble des nom d'episode de serie 
        for file in files:
            
            file = normaliseMot(listeFonctionNormaliseTitreEpisodeSerie(),file)
            file = filtreMot(listeFonctionFiltreTitreEpisodeSerie(),file)
            mot.append(file)

        #Parcours l'ensemble des saision d'une serie
        for directorie in directories:

            chemin = chemin_sous_repertoire_racine + "/" + directorie

            #Parcours les nom d'episode d'une saison
            for donnerEpisode in os.listdir(chemin) :

                #lecture des sous titres d'un episode pour extraire les donnees
                listeFichier = extractionDonnerFichierEpisode(chemin + "/" + donnerEpisode)
                for donner in listeFichier :
                    mot.append(donner)
                
            directorie = normaliseMot(listeFonctionNormaliseTitreSerie(),directorie)
            #directorie = filtreMot(listeFonctionNormaliseTitreSerie(),directorie)
            mot.append(directorie)
         
    return mot

def extractionDonnerFichierEpisode(chemin) :
     """
    Fonction qui va extraire les sous titres d'un episodes d'une series.
    
    param : chemin (string) -> chemin vers le fichier qui contient les sous titres
    return : liste[string] -> liste de tous les sous titres d'un épisodes d'une série
    """
    fichier = open(chemin, "r")
    mot = []
    lignes = fichier.readlines()
    for str in lignes:
        str = normaliseMot(listeFonctionNormaliseDonnerEpisodeSerie(),str)
        str = filtreMot(listeFonctionFiltreDonnerEpisodeSerie(),str)
        mot.append(str)

    return mot 

In [85]:
#################### TRADUCTION ####################

def traduitDonner(string):
    """
    Fonction qui va traduire les titres des séries et épisodes en une seule langue.
    
    param : string -> chaine de caractere qui corresponds a un titres d'une serie ou d'un épisode.
    return : string -> chaine de caractere traduit en une unique langue.
    """
    return string

In [34]:
#################### NORMALISE ####################

def normaliseMot(listeFonction,str):
    """
    Fonction qui va normaliser une donner.
    Cette fonction va normaliser une donner en fonction de notre liste de fonction a normaliser
    
    param : liste[fonction(string)] : liste de fonction qu'on souhaite appliquer a un mot
    param : str : string -> chaine de caractere a normaliser
    return : liste[string] -> chaine de caractere normaliser.
    """
    for fonctionNormalise in listeFonction :
        str = fonctionNormalise(str)
    return str

def listeFonctionNormaliseTitreSerie():
    """
    Retourne une liste de fonction qu'on souhaite appliquer a un titre d'une serie
    return liste[fonction(string)]
    """
    return [supprimePremierChiffreSerie,supprimeAccent,minusculeString,supprimePonctuation]

def listeFonctionNormaliseSaisonSerie():
    """
    Retourne une liste de fonction qu'on souhaite appliquer a un titre d'une saison d'une serie
    return liste[fonction(string)]
    """
    return [supprimeAccent,minusculeString,supprimePonctuation]

def listeFonctionNormaliseTitreEpisodeSerie():
    """
    Retourne une liste de fonction qu'on souhaite appliquer a un titre d'un episode d'une serie
    return liste[fonction(string)]
    """
    return [supprimePremierChiffreEpisodeSerie,supprimeAccent,minusculeString,supprimeExtension,supprimePonctuation,radicalMot]

def listeFonctionNormaliseDonnerEpisodeSerie():
    """
    Retourne une liste de fonction qu'on souhaite appliquer a un titre d'un episode d'une serie
    return liste[fonction(string)]
    """
    return [supprimeAccent,minusculeString,supprimePonctuation,radicalMot]

def supprimeAccent(str):
    """
    Fonction qui supprime les accents de notre str.

    param : str : string ->  chaine de caractere qu'on souhaite supprimer les accents.
    return string : chaine de caractere sans accent
    """
    return unicodedata.normalize('NFKD', str).encode('ASCII', 'ignore').decode("utf-8")

def minusculeString(str):
    """
    Fonction qui rends en minuscule notre str.

    param : str : string ->  chaine de caractere qu'on souhaite mettre en minuscule.
    return string : chaine de caractere en minuscule
    """
    return str.lower()

def supprimeExtension(str):
    """
    Fonction qui supprime l'extension de notre str

    param : str : string -> chaine de caractere qu'on souhaite supprimer l'extension.
    return string : chaine de caractere sans extenstion
    """
    #appliquer cette fonction avant supprimePonctuation
    return "".join(str.split(".")[0:-1]) #slicing pour supprimer l'extension .txt


def supprimePonctuation(str):
    """
    Fonction qui supprime les caracteres de ponctuation de notre str

    param : str : string -> chaine de caractere qu'on souhaite supprimer la ponctuation.
    return : string -> chaine de caractere sans ponctuation.
    Attention la chaine de caractere pourra etre composer de plusieurs mot composer par des espaces.
    """
    for ponctuations in string.punctuation :
        str = str.replace(ponctuations, " ")
    return supprimeChaineVide(str.split(" "))

def supprimeChaineVide(liste):
    """
    Fonction qui supprime les chaines vide de notre liste.

    param : liste[string] -> liste composer de chaine de caractere qu'on souhaite filtrer les chaines
    de caractere vide.
    return : string : chaine de caractere composer des differentes string de notre liste separer par un 
    unique espace entre chaque mot.
    """
    temp = []
    for i in liste :
        if i != "":
            temp.append( i + " ")
    return "".join(temp)

def supprimePremierChiffreSerie(str):
    """
    Fonction qui supprime le premier chiffre de  notre str. 
    Cette fonction est particuliere a la normalisation des titres des series

    param : string -> chaine de caractere qu'on souhaite extraire la racine.
    return : string : radical de notre chaine de caractere donner en parametre. 
    """
    temp = []
    liste = str.split("___")
    for i in liste[1:] :
        temp.append(i + " ")
    return "".join(temp)

def supprimePremierChiffreEpisodeSerie(str):
    """
    Fonction qui supprime le premier chiffre de  notre str. 
    Cette fonction est particuliere a la normalisation des titres des series

    param : string -> chaine de caractere qu'on souhaite extraire la racine.
    return : string : radical de notre chaine de caractere donner en parametre. 
    """
    temp = []
    liste = str.split("__")
    for i in liste[1:] :
        temp.append(i + " ")
    return "".join(temp)

def radicalMot(str) :
    """
    Fonction qui radicalise notre str c'est a dire garde uniquement la racine du mot.

    param : string -> chaine de caractere qu'on souhaite extraire la racine.
    return : string : radical de notre chaine de caractere donner en parametre. 
    """
    stemmer = SnowballStemmer("english")
    return stemmer.stem(str)
    


In [100]:

#################### FILTRE ####################

def filtreMot(listeFonction,str) :
    """
    Fonction qui va filtrer les donner.
 
    param : liste[funtion(string)] -> liste de function a un argumnt de type string qui filtre une chaine de caractere 
    return : string -> chaine de caractere filtrer en fonction des differentes fontion de filtrage qu'on souhaite       appliquer.
    """
    for fonctionFiltre in listeFonction :
        str = fonctionFiltre(str)
        if str == " ":
            return " "
    return str

def listeFonctionFiltreTitreSerie():
    """
    Retourne une liste de fonction qu'on souhaite appliquer a un titre d'une serie
    return liste[fonction(string)]
    """
    return []

def listeFonctionFiltreSaisonSerie():
    """
    Retourne une liste de fonction qu'on souhaite appliquer a un titre d'une saison d'une serie
    return liste[fonction(string)]
    """
    return []

def listeFonctionFiltreTitreEpisodeSerie():
    """
    Retourne une liste de fonction qu'on souhaite appliquer a un titre d'un episode d'une serie
    return liste[fonction(string)]
    """
    return [filtreNombre,filtreMotInutile]

def listeFonctionFiltreDonnerEpisodeSerie():
    """
    Retourne une liste de fonction qu'on souhaite appliquer a un titre d'un episode d'une serie
    return liste[fonction(string)]
    """
    return [filtreNombre,filtreAntiSlashN]

def filtreNombre(str):
    """
    Fonction qui va filtrer les nombres de notre str donné en parametre.
 
    param : string-> string sur lesquel on souhaite filtre les nombres.
    return : string -> chaine de caractere filtrer.
    """
    for chiffre in ['0','1','2','3','4','5','6','7','8',"9"] :
        str = str.replace(chiffre, " ")
    return supprimeChaineVide(str.split(" "))

def filtreAntiSlashN(str):
    """
    Fonction qui va filtrer les nombres de notre str donné en parametre.
 
    param : string-> string sur lesquel on souhaite filtre les nombres.
    return : string -> chaine de caractere filtrer.
    """
    for lettre in ['\n'] :
        str = str.replace(lettre, " ")
    return supprimeChaineVide(str.split(" "))


def filtreMotInutile(str):
    #grap ou info
    if str == "grab" or str == "info" :
        return " "
    return str

def filtreChaineVide(liste):
    """
    Fonction qui va filtrer les chaines de caractere vide.
 
    param : liste[string]-> string qui contient tous les mots a filtrer.
    return : liste[string] -> chaine de caractere filtrer.
    """
    resultat = []
    for i in liste :
        temp = i.split(" ")
        for k in temp :
            if k not in [""]:
                resultat.append(k)
    
    return resultat
        



In [15]:

#################### AUTRE ####################

def calcule_frequence_serie(liste):
    dic = dict()
    
    for i in liste :
        if i in dic :
            dic[i] += 1
        else :
            dic[i] = 1
    
    return dic

In [103]:
### TEST
extractionSeries(".")

lost
help
me
somebody
help
me
out
somebody
help
me
over
here
somebody
help
me
out
oh
my
god
walt
walt
stay
away
from
the
gas
stay
there
help
help
somebody
help
me
oh
my
leg
hey
get
over
here
give
me
a
hand
you
come
on
come
over
here
give
me
a
hand
on
the
count
of
three
one
two
three
help
please
help
me
help
me
please
help
me
get
him
out
of
here
get
him
away
from
the
engine
get
him
out
of
here
help
me
please
help
me
i
m
having
contractions
how
many
months
pregnant
are
you
only
eight
months
how
far
apart
are
they
coming
i
don
t
know
a
few
just
happened
hey
get
away
from
there
listen
to
me
look
at
me
you
re
gonna
be
okay
do
you
understand
me
but
you
have
to
sit
absolutely
still
hey
you
come
here
i
need
you
to
get
this
woman
away
from
these
fumes
take
her
there
stay
with
her
if
her
contractions
occur
closer
than
three
minutes
call
me
you
gotta
be
kidding
me
i
ll
be
right
back
thank
you
hey
what
s
your
name
jack
stop
her
head
s
not
tilted
back
enough
you
re
blowing
into
her
stomach
you
sure

['lost',
 'help',
 'me',
 'somebody',
 'help',
 'me',
 'out',
 'somebody',
 'help',
 'me',
 'over',
 'here',
 'somebody',
 'help',
 'me',
 'out',
 'oh',
 'my',
 'god',
 'walt',
 'walt',
 'stay',
 'away',
 'from',
 'the',
 'gas',
 'stay',
 'there',
 'help',
 'help',
 'somebody',
 'help',
 'me',
 'oh',
 'my',
 'leg',
 'hey',
 'get',
 'over',
 'here',
 'give',
 'me',
 'a',
 'hand',
 'you',
 'come',
 'on',
 'come',
 'over',
 'here',
 'give',
 'me',
 'a',
 'hand',
 'on',
 'the',
 'count',
 'of',
 'three',
 'one',
 'two',
 'three',
 'help',
 'please',
 'help',
 'me',
 'help',
 'me',
 'please',
 'help',
 'me',
 'get',
 'him',
 'out',
 'of',
 'here',
 'get',
 'him',
 'away',
 'from',
 'the',
 'engine',
 'get',
 'him',
 'out',
 'of',
 'here',
 'help',
 'me',
 'please',
 'help',
 'me',
 'i',
 'm',
 'having',
 'contractions',
 'how',
 'many',
 'months',
 'pregnant',
 'are',
 'you',
 'only',
 'eight',
 'months',
 'how',
 'far',
 'apart',
 'are',
 'they',
 'coming',
 'i',
 'don',
 't',
 'know',
 'a